In [1]:

import reflexgenerator.sources
from reflexgenerator.sources import (
    Register, Mask, PinMap_from_json, Metadata, Collection)
from reflexgenerator.io import load

from pathlib import Path
import pandas as pd
from reflexgenerator.generator.xref import UidReference, make_anchor, create_link, filter_refs_by_type


In [2]:
fw_target = r"schema\common.yml"
io_target = r"schema\ios.yml"

In [3]:
device = load(fw_target)
ios = load(io_target)

In [6]:
metadata = Metadata(
    **{"device": device["device"],
    "whoAmI": device["whoAmI"],
    "architecture": device["architecture"],
    "firmwareVersion": device["firmwareVersion"],
    "hardwareTargets": device["hardwareTargets"]})
print(metadata)


KeyError: 'device'

In [4]:
# build masks
if "bitMasks" in device:
    bitMasks = Collection([Mask.from_json(mask) for mask in device["bitMasks"].items() if mask is not None])
    bitMasks_df = pd.DataFrame([mask.to_dict() for mask in bitMasks])
    bitMasks_df["name"] = bitMasks_df["uid"].apply(lambda x: (x.render_pointer()))
    bitMasks_df["bits"] = bitMasks_df["bits"].apply(lambda x: [bit.value for bit in x if bit is not None] if x is not None else None)
    bitMasks_df.head(5)
else:
    bitMasks = None
    bitMasks_df = pd.DataFrame()

if "groupMasks" in device:
    groupMasks = Collection([Mask.from_json(mask) for mask in device["groupMasks"].items() if mask is not None])
    groupMasks_df = pd.DataFrame([mask.to_dict() for mask in groupMasks])
    groupMasks_df["name"] = groupMasks_df["uid"].apply(lambda x: (x.render_pointer()))
    groupMasks_df["values"] = groupMasks_df["values"].apply(lambda x: [bit.value for bit in x if bit is not None] if x is not None else None)
    groupMasks_df.head(5)
else:
    groupMasks = None
    groupMasks_df = pd.DataFrame()

## Create deep copies to manipulate
bitMasks_df_copy = bitMasks_df.copy() if groupMasks_df is not None else pd.DataFrame()
groupMasks_df_copy = groupMasks_df.copy() if groupMasks_df is not None else pd.DataFrame()


KeyError: 'values'

In [5]:
# build registers
if "registers" in device:
    regs = Collection([Register.from_json(reg) for reg in device["registers"].items() if reg is not None])
    register_df = pd.DataFrame([reg.to_dict() for reg in regs])
    register_df["name"] = register_df["uid"].apply(lambda x: (x.render_pointer()))
    register_df.head(5)
else:
    regs = None
    register_df = pd.DataFrame()


register_df_copy = register_df.copy()

for i in register_df_copy.index:
    if register_df_copy.at[i, 'maskType'] is not None:
        if isinstance(register_df_copy.at[i, 'maskType'][0], reflexgenerator.sources.Mask):
            register_df_copy.at[i, 'maskType'] = [x.uid.render_pointer() for x in register_df_copy.at[i, 'maskType']]
    if register_df_copy.at[i, 'payloadSpec'] is not None:
            if isinstance(register_df_copy.at[i, 'payloadSpec'][0], reflexgenerator.sources.PayloadMember):
                register_df_copy.at[i, 'payloadSpec'] = [x.uid.render_pointer() for x in register_df_copy.at[i, 'payloadSpec']]

KeyError: 'Specified mask, OperationMode, has not been defined.'

In [10]:
#Build payload members
payloadMembers = Collection([entry.parent for entry in filter_refs_by_type(reflexgenerator.sources.PayloadMember).values()])
payloadMembers_df = pd.DataFrame([payloadMember.to_dict() for payloadMember in payloadMembers])
payloadMembers_df["name"] = payloadMembers_df["uid"].apply(lambda x: (x.render_pointer()))

#deep copy
payloadMembers_df_copy = payloadMembers_df.copy() if payloadMembers_df is not None else pd.DataFrame()

In [11]:
#Build IOs
pinMapping = Collection([PinMap_from_json(pinmap) for pinmap in ios.items() if pinmap is not None])
pinMapping_df = pd.DataFrame([pinmap.to_dict() for pinmap in pinMapping])
pinMapping_df["name"] = pinMapping_df["uid"].apply(lambda x: (x.render_pointer()))


In [13]:
txt = f"""

# Device

{metadata.format_dict()}
--------

# Registers

## Summary table
{reflexgenerator.generator.format_table(register_df_copy)}

## Technical documentation
{"".join([reg.format_dict() for reg in regs])}
--------

# BitMasks

## Summary table
{reflexgenerator.generator.format_table(bitMasks_df_copy)}

## Technical documentation
{"".join([mask.format_dict() for mask in bitMasks])}
--------
# PayloadMembers

## Summary table
{reflexgenerator.generator.format_table(payloadMembers_df_copy)}

## Technical documentation
{"".join([payloadMember.format_dict() for payloadMember in payloadMembers])}

# IOs
{reflexgenerator.generator.format_table(pinMapping_df)}

## Technical documentation
{"".join([pin.format_dict() for pin in pinMapping])}


## References
{reflexgenerator.sources.PayloadType.format_anchor_references()}

{reflexgenerator.sources.AccessType.format_anchor_references()}

{reflexgenerator.sources.VisibilityType.format_anchor_references()}

{reflexgenerator.sources.VolatilityType.format_anchor_references()}

{reflexgenerator.sources.MaskCategory.format_anchor_references()}

{reflexgenerator.sources.DirectionType.format_anchor_references()}

{reflexgenerator.sources.InputPinModeType.format_anchor_references()}

{reflexgenerator.sources.TriggerModeType.format_anchor_references()}

{reflexgenerator.sources.InterruptPriorityType.format_anchor_references()}

{reflexgenerator.sources.OutputPinModeType.format_anchor_references()}

{reflexgenerator.sources.InitialStateType.format_anchor_references()}

"""

In [14]:
with open("demo_docs\documentation.md", "w") as text_file:
    text_file.write(txt)

import markdown
with open("demo_docs\documentation.html", "w") as text_file:
    text_file.write(markdown.markdown(txt, tab_length=4, extensions=['extra', 'smarty', 'sane_lists']))